In [1]:
import csv
import json
import sys
import pandas as pd
import os

In [ ]:
def createBaskets(sourceFile, destFile):
    tsv = csv.reader(open(sourceFile, encoding="utf-8"), delimiter="\t")
    print(sys.getsizeof(tsv))
    c=0
    keys = []
    baskets = []
    items = []
    for row in tsv:
        if row[3]=="actor" or row[3]=="actress":
            
            if row[2] not in items:
                items.append(row[2])
            pos = items.index(row[2])
            
            if row[0] in keys:
                baskets[keys.index(row[0])] += [pos]
            else:
                keys += [row[0]]
                baskets.append([pos])
            if c==1000:
                break;
            c = c+1
#     return keys, baskets
    with open(destFile+".csv","w") as f:
        wr = csv.writer(f,delimiter=";")
        for el in baskets:
            wr.writerow(el)
    with open("actorIdToInt.csv","w") as f:
        wr = csv.writer(f,delimiter=";")
        for el in items:
            wr.writerow([el])
    #json.dump(baskets, open(destFile+'.json', 'w'))
    #json.dump(keys, open('actorIdToInt.json', 'w'))

In [ ]:
createBaskets("data.tsv", "preprocessed")

96


In [ ]:
loaded = json.load(open("preprocessed.json", encoding="utf-8"))
print(loaded)

[[0, 1], [2, 3], [4], [5, 6, 7], [8], [9, 10], [11, 12], [13], [14, 15, 16, 17], [18], [19], [20], [21], [21], [21], [21], [22], [23, 21], [19], [23, 24, 21], [19], [19], [19], [19], [19], [19], [25, 26, 27], [21], [28, 29, 21, 30], [21], [31], [23], [21], [32, 33, 34, 35, 36], [37, 38], [39, 40], [41], [37], [39, 40], [21], [23, 21], [41], [39], [21], [42, 43, 44, 23, 45], [23, 21], [23, 21], [21], [21], [46, 47, 48, 49], [37, 50], [21], [21], [51], [52, 53, 54], [31], [31], [31], [55], [41], [56], [57], [58, 59, 60], [41], [61, 56], [62, 63], [51, 64], [37, 38], [65], [66, 67], [57], [57], [68, 69, 70, 71, 72, 73], [74], [74], [31], [75, 76], [77, 78, 79], [21], [75, 76], [80], [81], [82, 83, 84, 85, 86, 87, 88], [75, 76], [31], [89], [90], [91, 92], [38], [93, 94, 95], [21], [74], [21, 96, 43, 97], [62, 63, 98, 99], [79], [79], [21], [31], [31], [100], [79], [61], [101, 102, 103, 104, 105, 106, 107], [21], [108, 109], [79], [110, 111, 112, 91], [37], [102], [21], [19], [21, 113, 43,

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz    
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip3 install -q findspark
!pip3 install -q pyspark
!pip3 install -q kaggle

from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets files "ashirwadsangwan/imdb-dataset"
!kaggle datasets download "ashirwadsangwan/imdb-dataset" -f "title.principals.tsv.gz"
!unzip "title.principals.tsv.gz.zip"
!rm "title.principals.tsv.gz.zip"
!gunzip -k "title.principals.tsv.gz"
!rm "title.principals.tsv.gz"

Saving kaggle.json to kaggle.json
kaggle.json
name                      size  creationDate         
-----------------------  -----  -------------------  
title.principals.tsv.gz  290MB  2019-11-18 18:36:47  
title.ratings.tsv.gz       5MB  2019-11-18 18:36:47  
title.akas.tsv.gz        158MB  2019-11-18 18:36:47  
name.basics.tsv.gz       180MB  2019-11-18 18:36:47  
title.basics.tsv.gz      108MB  2019-11-18 18:36:47  
 96% 276M/287M [00:02<00:00, 128MB/s]
100% 287M/287M [00:03<00:00, 98.7MB/s]
Archive:  title.principals.tsv.gz.zip
  inflating: title.principals.tsv.gz  


In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

home = os.environ["SPARK_HOME"]
os.environ["PYTHONPATH"] = home+"/python"+home+"/python/lib/py4j-0.10.9-src.zip:"+os.environ["PYTHONPATH"]

In [4]:
import findspark
from pyspark.sql import SparkSession
import pyspark
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [25]:
sourceFile = 'title.principals.tsv'

data = spark.read.options(delimiter = "\t", header=True).csv(sourceFile)

rdd_result = data.rdd.map(lambda row: (row["nconst"], 0) if row["category"] == 'actress' or row["category"] == 'actor' else (-1, 0)).reduceByKey(lambda a, b: 0)
res = rdd_result.take(100)

In [36]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
df = sqlContext.createDataFrame(res, ['item', 'value']).select('item')
df.coalesce(1).write.format('com.databricks.spark.csv').save("folder.csv")

In [37]:
!mv folder.csv/*.csv actorIdToInt.csv

In [38]:
!rm -rf actorIdToInt.csv/

Create preprocessed file

In [7]:
sourceFile = 'title.principals.tsv'

data = spark.read.options(delimiter = "\t", header=True).csv(sourceFile)

def filter_actors(row):
    if row["category"] == 'actress' or row["category"] == 'actor':
        return (row["tconst"], [row["nconst"]])

rdd_result = data.rdd.map(lambda row: (row["tconst"], [row["nconst"]]) if row["category"] == 'actress' or row["category"] == 'actor' else (-1, [])).reduceByKey(lambda a, b: a + b)
res = rdd_result.take(100)

In [19]:
df = sqlContext.createDataFrame(res, ['movie_id', 'actors']).select('actors')

In [24]:
df.write.format('com.databricks.spark.csv').save("preprocessed.csv")

AnalysisException: ignored

In [15]:
def toCSVLine(data):
  return ','.join(str(d[1]) for d in data)

lines = labelsAndPredictions.map(toCSVLine)
lines.saveAsTextFile('hdfs://my-node:9000/tmp/labels-and-predictions.csv')

Py4JJavaError: ignored